In [84]:
# import dependencies
import numpy as np
import pandas as pd
from pathlib import Path

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [85]:
# read in the csv files and assign to training and testing dataframes
train_df = pd.read_csv(Path('Resources/2019loans.csv'))
test_df = pd.read_csv(Path('Resources/2020Q1loans.csv'))

In [86]:
# show all columns and display training dataframe 
pd.set_option('display.max_columns', None)
train_df

Unnamed: 0   index  loan_amnt  int_rate  installment home_ownership  \
0           57107   57107    13375.0    0.1797       483.34       MORTGAGE   
1          141451  141451    21000.0    0.1308       478.68       MORTGAGE   
2          321143  321143    20000.0    0.1240       448.95       MORTGAGE   
3           11778   11778     3000.0    0.1240       100.22           RENT   
4          169382  169382    30000.0    0.1612      1056.49       MORTGAGE   
...           ...     ...        ...       ...          ...            ...   
12175      354912  354912    19975.0    0.2565       801.09           RENT   
12176      354944  354944    15000.0    0.1774       540.34           RENT   
12177      354973  354973     3600.0    0.1862       131.28           RENT   
12178      355002  355002    15000.0    0.0881       475.68       MORTGAGE   
12179      355307  355307    15000.0    0.1774       540.34           RENT   

       annual_inc verification_status loan_status pymnt_plan    dti  \
0        223000.0        Not Verified    low_risk          n  29.99   
1        123000.0     Source Verified    low_risk          n  11.26   
2        197000.0     Source Verified    low_risk          n  11.28   
3         45000.0        Not Verified    low_risk          n  18.08   
4        133000.0     Source Verified    low_risk          n  27.77   
...           ...                 ...         ...        ...    ...   
12175     28000.0        Not Verified   high_risk          n  28.42   
12176     50000.0            Verified   high_risk          n  23.43   
12177     60000.0        Not Verified   high_risk          n  28.80   
12178     62000.0     Source Verified   high_risk          n  11.44   
12179     54080.0        Not Verified   high_risk          n  21.66   

       delinq_2yrs  inq_last_6mths  open_acc  pub_rec  revol_bal  total_acc  \
0              0.0             0.0      15.0      0.0    39728.0       38.0   
1              2.0             0.0      16.0      0.0     9585.0       20.0   
2              0.0             0.0      12.0      0.0    16708.0       36.0   
3              0.0             0.0      12.0      1.0     8809.0       19.0   
4              0.0             2.0      13.0      0.0    65420.0       51.0   
...            ...             ...       ...      ...        ...        ...   
12175          0.0             0.0      15.0      0.0     9679.0       32.0   
12176          4.0             0.0      16.0      0.0     3193.0       27.0   
12177          0.0             1.0      14.0      0.0    11804.0       35.0   
12178          0.0             0.0       5.0      0.0      157.0       15.0   
12179          1.0             1.0       5.0      0.0     8349.0       11.0   

      initial_list_status  out_prncp  out_prncp_inv  total_pymnt  \
0                       w   12804.66       12804.66       953.33   
1                       w   15152.97       15152.97     10037.02   
2                       w   15724.23       15724.23      6971.52   
3                       w    2930.77        2930.77        81.63   
4                       w   20778.03       20778.03     14763.99   
...                   ...        ...            ...          ...   
12175                   w   16305.16       16305.16      7181.35   
12176                   w   12409.98       12409.98      5200.43   
12177                   w    2670.43        2670.43      1571.64   
12178                   w   10827.99       10827.99      5225.14   
12179                   f   10616.52       10616.52      7209.64   

       total_pymnt_inv  total_rec_prncp  total_rec_int  total_rec_late_fee  \
0               953.33           570.34         382.99                 0.0   
1             10037.02          5847.03        4189.99                 0.0   
2              6971.52          4275.77        2695.75                 0.0   
3                81.63            69.23          12.40                 0.0   
4             14763.99          9221.97        5542.02       

In [87]:
test_df

Unnamed: 0  index  loan_amnt  int_rate  installment home_ownership  \
0          67991  67991    40000.0    0.0819       814.70       MORTGAGE   
1          25429  25429     6000.0    0.1524       208.70           RENT   
2          38496  38496     3600.0    0.1695       128.27           RENT   
3          19667  19667    20000.0    0.1524       478.33           RENT   
4          37505  37505     3600.0    0.1240       120.27           RENT   
...          ...    ...        ...       ...          ...            ...   
4697       77282  77282    30000.0    0.1240       673.42           RENT   
4698       77291  77291    24000.0    0.0756       747.22           RENT   
4699       77292  77292    10000.0    0.2305       387.36           RENT   
4700       77297  77297     8000.0    0.1862       205.86           RENT   
4701       77304  77304    30000.0    0.2055      1123.34           RENT   

      annual_inc verification_status loan_status pymnt_plan    dti  \
0       140000.0        Not Verified    low_risk          n  19.75   
1        55000.0        Not Verified    low_risk          n  11.52   
2        42000.0        Not Verified    low_risk          n   6.74   
3       100000.0        Not Verified    low_risk          n  12.13   
4        50000.0        Not Verified    low_risk          n  16.08   
...          ...                 ...         ...        ...    ...   
4697    140480.0     Source Verified   high_risk          n  15.74   
4698     50000.0        Not Verified   high_risk          n  26.81   
4699     33000.0            Verified   high_risk          n  38.51   
4700     38000.0     Source Verified   high_risk          n  16.36   
4701    180000.0     Source Verified   high_risk          n  12.06   

      delinq_2yrs  inq_last_6mths  open_acc  pub_rec  revol_bal  total_acc  \
0             0.0             1.0      18.0      0.0     9471.0       45.0   
1             2.0             0.0       8.0      0.0     1280.0       12.0   
2             0.0             0.0       6.0      0.0     4757.0        6.0   
3             0.0             2.0       7.0      0.0    12731.0       21.0   
4             0.0             3.0       6.0      0.0    10413.0        6.0   
...           ...             ...       ...      ...        ...        ...   
4697          0.0             0.0      20.0      0.0    23215.0       32.0   
4698          0.0             0.0       9.0      0.0      459.0       10.0   
4699          0.0             2.0       7.0      0.0     6342.0       13.0   
4700          0.0             1.0       8.0      1.0    11636.0       20.0   
4701          0.0             0.0       8.0      0.0     4771.0       24.0   

     initial_list_status  out_prncp  out_prncp_inv  total_pymnt  \
0                      w   33249.95       33249.95      9740.00   
1                      w       0.00           0.00      6185.59   
2                      w       0.00           0.00      3603.73   
3                      w   17624.06       17624.06      4766.37   
4                      w       0.00           0.00      3602.48   
...                  ...        ...            ...          ...   
4697                   f   26196.18       26196.18      6713.53   
4698                   w       0.00           0.00      4526.54   
4699                   f       0.00           0.00      2685.91   
4700                   w       0.00           0.00      2075.44   
4701                   f   28770.38       28770.38      5582.45   

      total_pymnt_inv  total_rec_prncp  total_rec_int  total_rec_late_fee  \
0             9740.00          6750.05        2989.95                 0.0   
1             6185.59          6000.00         185.59                 0.0   
2             3603.73          3600.00           3.73                 0.0   
3             4766.37          2375.94        2390.43                 0.0   
4             3602.48          3600.00           2.48                 0.0   
...               ...              ...          

In [88]:
# Convert categorical data to numeric and separate target feature for training data
y_train = train_df["loan_status"]
x_train = train_df.drop(columns = ["loan_status"])
x_train = pd.get_dummies(x_train)
x_train

Unnamed: 0   index  loan_amnt  int_rate  installment  annual_inc  \
0           57107   57107    13375.0    0.1797       483.34    223000.0   
1          141451  141451    21000.0    0.1308       478.68    123000.0   
2          321143  321143    20000.0    0.1240       448.95    197000.0   
3           11778   11778     3000.0    0.1240       100.22     45000.0   
4          169382  169382    30000.0    0.1612      1056.49    133000.0   
...           ...     ...        ...       ...          ...         ...   
12175      354912  354912    19975.0    0.2565       801.09     28000.0   
12176      354944  354944    15000.0    0.1774       540.34     50000.0   
12177      354973  354973     3600.0    0.1862       131.28     60000.0   
12178      355002  355002    15000.0    0.0881       475.68     62000.0   
12179      355307  355307    15000.0    0.1774       540.34     54080.0   

         dti  delinq_2yrs  inq_last_6mths  open_acc  pub_rec  revol_bal  \
0      29.99          0.0             0.0      15.0      0.0    39728.0   
1      11.26          2.0             0.0      16.0      0.0     9585.0   
2      11.28          0.0             0.0      12.0      0.0    16708.0   
3      18.08          0.0             0.0      12.0      1.0     8809.0   
4      27.77          0.0             2.0      13.0      0.0    65420.0   
...      ...          ...             ...       ...      ...        ...   
12175  28.42          0.0             0.0      15.0      0.0     9679.0   
12176  23.43          4.0             0.0      16.0      0.0     3193.0   
12177  28.80          0.0             1.0      14.0      0.0    11804.0   
12178  11.44          0.0             0.0       5.0      0.0      157.0   
12179  21.66          1.0             1.0       5.0      0.0     8349.0   

       total_acc  out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  \
0           38.0   12804.66       12804.66       953.33           953.33   
1           20.0   15152.97       15152.97     10037.02         10037.02   
2           36.0   15724.23       15724.23      6971.52          6971.52   
3           19.0    2930.77        2930.77        81.63            81.63   
4           51.0   20778.03       20778.03     14763.99         14763.99   
...          ...        ...            ...          ...              ...   
12175       32.0   16305.16       16305.16      7181.35          7181.35   
12176       27.0   12409.98       12409.98      5200.43          5200.43   
12177       35.0    2670.43        2670.43      1571.64          1571.64   
12178       15.0   10827.99       10827.99      5225.14          5225.14   
12179       11.0   10616.52       10616.52      7209.64          7209.64   

       total_rec_prncp  total_rec_int  total_rec_late_fee  recoveries  \
0               570.34         382.99                 0.0         0.0   
1              5847.03        4189.99                 0.0         0.0   
2              4275.77        2695.75                 0.0         0.0   
3                69.23          12.40                 0.0         0.0   
4              9221.97        5542.02                 0.0         0.0   
...                ...            ...                 ...         ...   
12175          3669.84        3511.51                 0.0         0.0   
12176          2590.02        2610.41                 0.0         0.0   
12177           929.57         642.07                 0.0         0.0   
12178          4172.01        1053.13                 0.0         0.0   
12179          4383.48        2826.16                 0.0         0.0   

       collection_recovery_fee  last_pymnt_amnt  collections_12_mths_ex_med  \
0                          0.0           483.34                         0.0   
1                          0.0           478.68                         0.0   
2                          0.0           448.95                         0.0   
3                          0.0            85.76                         1.0   
4                   

In [89]:
# Convert categorical data to numeric and separate target feature for testing data
y_test = test_df["loan_status"]
x_test = test_df.drop(columns = ["loan_status"])
x_test = pd.get_dummies(x_test)
x_test

Unnamed: 0  index  loan_amnt  int_rate  installment  annual_inc    dti  \
0          67991  67991    40000.0    0.0819       814.70    140000.0  19.75   
1          25429  25429     6000.0    0.1524       208.70     55000.0  11.52   
2          38496  38496     3600.0    0.1695       128.27     42000.0   6.74   
3          19667  19667    20000.0    0.1524       478.33    100000.0  12.13   
4          37505  37505     3600.0    0.1240       120.27     50000.0  16.08   
...          ...    ...        ...       ...          ...         ...    ...   
4697       77282  77282    30000.0    0.1240       673.42    140480.0  15.74   
4698       77291  77291    24000.0    0.0756       747.22     50000.0  26.81   
4699       77292  77292    10000.0    0.2305       387.36     33000.0  38.51   
4700       77297  77297     8000.0    0.1862       205.86     38000.0  16.36   
4701       77304  77304    30000.0    0.2055      1123.34    180000.0  12.06   

      delinq_2yrs  inq_last_6mths  open_acc  pub_rec  revol_bal  total_acc  \
0             0.0             1.0      18.0      0.0     9471.0       45.0   
1             2.0             0.0       8.0      0.0     1280.0       12.0   
2             0.0             0.0       6.0      0.0     4757.0        6.0   
3             0.0             2.0       7.0      0.0    12731.0       21.0   
4             0.0             3.0       6.0      0.0    10413.0        6.0   
...           ...             ...       ...      ...        ...        ...   
4697          0.0             0.0      20.0      0.0    23215.0       32.0   
4698          0.0             0.0       9.0      0.0      459.0       10.0   
4699          0.0             2.0       7.0      0.0     6342.0       13.0   
4700          0.0             1.0       8.0      1.0    11636.0       20.0   
4701          0.0             0.0       8.0      0.0     4771.0       24.0   

      out_prncp  out_prncp_inv  total_pymnt  total_pymnt_inv  total_rec_prncp  \
0      33249.95       33249.95      9740.00          9740.00          6750.05   
1          0.00           0.00      6185.59          6185.59          6000.00   
2          0.00           0.00      3603.73          3603.73          3600.00   
3      17624.06       17624.06      4766.37          4766.37          2375.94   
4          0.00           0.00      3602.48          3602.48          3600.00   
...         ...            ...          ...              ...              ...   
4697   26196.18       26196.18      6713.53          6713.53          3803.82   
4698       0.00           0.00      4526.54          4526.54          1195.80   
4699       0.00           0.00      2685.91          2685.91          1534.41   
4700       0.00           0.00      2075.44          2075.44           185.00   
4701   28770.38       28770.38      5582.45          5582.45          1229.62   

      total_rec_int  total_rec_late_fee  recoveries  collection_recovery_fee  \
0           2989.95                 0.0        0.00                   0.0000   
1            185.59                 0.0        0.00                   0.0000   
2              3.73                 0.0        0.00                   0.0000   
3           2390.43                 0.0        0.00                   0.0000   
4              2.48                 0.0        0.00                   0.0000   
...             ...                 ...         ...                      ...   
4697        2909.71                 0.0        0.00                   0.0000   
4698         288.56                 0.0     3042.18                 547.5924   
4699        1151.50                 0.0        0.00                   0.0000   
4700         416.03                 0.0     1474.41                 265.3938   
4701        4352.83                 0.0        0.00                   0.0000   

      last_pymnt_amnt  collections_12_mths_ex_med  policy_code  \
0              814.70                         0.0          1.0   
1             5773.27                     

In [90]:
# add missing dummy variables to testing set
for column in x_train.columns:
    if column not in x_test.columns:
        x_test[column] = 0

## Predictions- Unscaled Data
I predict the Random Forest Classifier will perform better than the Logistic Regression because these datasets consist of both numerical and categorial data, and Random Forest models tend to work better on datasets with categorical data. Logistic Regression models are faster, and usually used when working with finance datasets, but Random Forest will probably outperform the regression because of the small size of this dataset. Random Forest generates predictions from multiple decision trees and then performs voting on the best prediction, which can be time-consuming, but because of the size and complexity of this dataset, processing time is not a factor.

In [91]:
# Train the Logistic Regression model on the unscaled data and print the model score
regression = LogisticRegression().fit(x_train, y_train)

print('Logistic Regression Model Training Score: ', regression.score(x_train,y_train) )
print('Logistic Regression Model Testing Score: ', regression.score(x_test,y_test) )

Logistic Regression Model Training Score:  0.6485221674876848
Logistic Regression Model Testing Score:  0.5253083794130158


C:\Users\amylb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [92]:
# Train a Random Forest Classifier model and print the model score
# Initial Test
random_forest = RandomForestClassifier(random_state=0)
random_forest.fit(x_train,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test,y_test))

Random Forest Classifier Training Score:  1.0
Random Forest Classifier Testing Score:  0.6146320714589536


In [93]:
# Train a Random Forest Classifier model and print the model score
# Adjusting random state parameter
random_forest = RandomForestClassifier(random_state=42)
random_forest.fit(x_train,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test,y_test))

Random Forest Classifier Training Score:  1.0
Random Forest Classifier Testing Score:  0.6305827307528711


In [94]:
# Train a Random Forest Classifier model and print the model score
# Adjusting random state and n_estimator parameters
random_forest = RandomForestClassifier(random_state=0, n_estimators=500)
random_forest.fit(x_train,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test,y_test))

Random Forest Classifier Training Score:  1.0
Random Forest Classifier Testing Score:  0.6152700978307103


In [95]:
# Train a Random Forest Classifier model and print the model score
# Adjusting only n_estimators parameter
random_forest = RandomForestClassifier(n_estimators=500)
random_forest.fit(x_train,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test,y_test))

Random Forest Classifier Training Score:  1.0
Random Forest Classifier Testing Score:  0.6065504040833688


In [96]:
# Train a Random Forest Classifier model and print the model score
# Adjusting max_depth parameter
random_forest = RandomForestClassifier(max_depth=5)
random_forest.fit(x_train,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test,y_test))

Random Forest Classifier Training Score:  0.7578817733990147
Random Forest Classifier Testing Score:  0.5663547426626967


## Prediction Results- Unscaled Data
As predicted, the Random Forest Classifier did perform better with a score of 0.61, but in the first model the training score was 1 which tells us that there is an overfitting problem. In order to fix this overfitting problem, I modify the parameters to see what would happen to the training score. Initially, I changed the random state, which did not impact the score. Subsquently, I tried adjusting the n_estimators to see the impact, but I think the dataset is too small for that parameter to be effective. Finally, when adjusting the max_depth I was able to see an impact on the training score. In the final result, the testing score stayed consistent around 0.61, but the training score fell to 0.76, which fixed the overfitting problem.

In [97]:
# Scale the data
scaler = StandardScaler().fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

## Predictions- Scaled Data
I think after scaling the data, the Logistic Regression will perform better than the previous test and the Random Forest Classifier will perform about the same. Tree-based algorithms are fairly insensitive to scaling of the features, so I predict there will not be much of a change with the Random Forest score. The Logisitic Regression could potentially outperform the Random Forest Classifier after scaling.

In [98]:
# Train the Logistic Regression model on the scaled data and print the model score

regression = LogisticRegression().fit(x_train_scaled, y_train)

print('Logistic Regression Model Training Score: ', regression.score(x_train_scaled,y_train) )
print('Logistic Regression Model Testing Score: ', regression.score(x_test_scaled,y_test) )

Logistic Regression Model Training Score:  0.713136288998358
Logistic Regression Model Testing Score:  0.7201190982560612


C:\Users\amylb\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [99]:
# Train a Random Forest Classifier model on the scaled data and print the model score

random_forest = RandomForestClassifier(random_state=0)
random_forest.fit(x_train_scaled,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train_scaled,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test_scaled,y_test))

Random Forest Classifier Training Score:  1.0
Random Forest Classifier Testing Score:  0.6150574223734581


In [100]:
# Train a Random Forest Classifier model on the scaled data and print the model score
# Testing adjusting of max_depth parameters

random_forest = RandomForestClassifier(max_depth=5)
random_forest.fit(x_train_scaled,y_train)


print('Random Forest Classifier Training Score: ', random_forest.score(x_train_scaled,y_train))
print('Random Forest Classifier Testing Score: ', random_forest.score(x_test_scaled,y_test))

Random Forest Classifier Training Score:  0.75935960591133
Random Forest Classifier Testing Score:  0.5710336027222459


## Prediction Results- Scaled Data
As predicted, the Random Forest Classifier was unaffected by the scaling of the data. In fact, after scaling the data and adjusting the max_depth parameters, the model performed worse than the Logisitic Regression model before the data was scaled. The Logisitic Regression did improve as a result of the scaling, and did outperform the Random Forest model with a score of 0.72.

In [102]:
# Confusion matrix for logistic regression
y_true = y_test
y_pred = regression.predict(x_test_scaled)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(confusion)

[[1242 1109]
 [ 207 2144]]


In [103]:
# Confusion matric for random forest classifier 
y_true_rf = y_test
y_pred_rf = random_forest.predict(x_test_scaled)
confusion = metrics.confusion_matrix(y_true_rf, y_pred_rf)
print(confusion)

[[ 472 1879]
 [ 138 2213]]
